## División de cursos mediante Algoritmos Genéticos

 **Contexto:**
En una escuela secundaria, 60 alumnos pasaron exitosamente el curso de ingreso.Ahora deben ser asignados a 3 cursos de 20 alumnos cada uno. Cada alumno eligió dos compañeros con quienes le gustaría estar en el mismo curso.


**Objetivo:**
Construir una asignacion que maximice la utilidad total del grupo.

**Reglas:**
- Cada curso debe tener exactamente 20 alumnos.
- Cada alumno debe pertenecer a un único curso.

**Definiciones de utilidad (fitness):**
- +1 punto por cada amigo elegido que este en el mismo curso (Osea + 2 si los 2 amigos están en el mismo curso)
- -3 puntos si los dos amigos terminan juntos en otro curso.
- -1 punto si el alumno y sus dos amigos terminan todos en cursos distintos.

In [125]:
# Listado de alumnos y sus preferencias
alumnos_amigos = {
    'Ana': ['Bruno', 'Camila'], 'Bruno': ['Ana', 'Diego'], 'Camila': ['Bruno', 'Elena'],
    'Diego': ['Camila', 'Francisco'], 'Elena': ['Diego', 'Gabriela'], 'Francisco': ['Elena', 'Hugo'],
    'Gabriela': ['Francisco', 'Isabel'], 'Hugo': ['Gabriela', 'Juan'], 'Isabel': ['Hugo', 'Karla'],
    'Juan': ['Isabel', 'Leonardo'], 'Karla': ['Juan', 'Mariana'], 'Leonardo': ['Karla', 'Nicolas'],
    'Mariana': ['Leonardo', 'Olivia'], 'Nicolas': ['Mariana', 'Pablo'], 'Olivia': ['Nicolas', 'Raul'],
    'Pablo': ['Olivia', 'Sofia'], 'Raul': ['Pablo', 'Tomas'], 'Sofia': ['Raul', 'Ursula'],
    'Tomas': ['Sofia', 'Valeria'], 'Ursula': ['Tomas', 'Walter'], 'Valeria': ['Ursula', 'Xavier'],
    'Walter': ['Valeria', 'Yara'], 'Xavier': ['Walter', 'Zoe'], 'Yara': ['Xavier', 'Andres'],
    'Zoe': ['Yara', 'Belen'], 'Andres': ['Zoe', 'Carlos'], 'Belen': ['Andres', 'Daniela'],
    'Carlos': ['Belen', 'Esteban'], 'Daniela': ['Carlos', 'Fernanda'], 'Esteban': ['Daniela', 'Gustavo'],
    'Fernanda': ['Esteban', 'Helena'], 'Gustavo': ['Fernanda', 'Ignacio'], 'Helena': ['Gustavo', 'Julieta'],
    'Ignacio': ['Helena', 'Kevin'], 'Julieta': ['Ignacio', 'Laura'], 'Kevin': ['Julieta', 'Mateo'],
    'Laura': ['Kevin', 'Natalia'], 'Mateo': ['Laura', 'Octavio'], 'Natalia': ['Mateo', 'Paula'],
    'Octavio': ['Natalia', 'Ricardo'], 'Paula': ['Octavio', 'Sabrina'], 'Ricardo': ['Paula', 'Tadeo'],
    'Sabrina': ['Ricardo', 'Ursula'], 'Tadeo': ['Sabrina', 'Victor'], 'Victor': ['Ursula', 'Ximena'],
    'Wanda': ['Victor', 'Yamila'], 'Ximena': ['Wanda', 'Zoe'], 'Yamila': ['Ximena', 'Ana'], 'Zoe': ['Yamila', 'Bruno'],
    'Beltran': ['Ana', 'Diego'], 'Catalina': ['Francisco', 'Gabriela'], 'Dario': ['Hugo', 'Isabel'],
    'Emilia': ['Leonardo', 'Mariana'], 'Fabian': ['Nicolas', 'Olivia'], 'Gisela': ['Pablo', 'Raul'],
    'Hernan': ['Sofia', 'Valeria'], 'Isidoro': ['Walter', 'Xavier'], 'Jimena': ['Yara', 'Andres'],
    'Horacio': ['Belen', 'Carlos'], 'Luz': ['Daniela', 'Esteban'], 'Martin': ['Fernanda', 'Gustavo']
}

**Entregables:**
- 1. Código en Python con comentarios.
- 2. Breve descripción sobre cómo definieron el genoma, la funcion de aptitud, el tamaño de la población inicial, la tasa de mutación y por qué eligieron un determinado método de selección de padres.
- 3. División de cursos que maximiza la utilidad total y valor de esa utilidad total.

**Bonus 1 (opcional):**
Implementar elitismo:
- Guardar el mejor individuo en cada generación y pasarlo directamente sin modificaciones.

**Bonus 2 (opcional):**
Por motivos personales los alumnos Xavier y Zoe deben compartir curso si o si con sus dos amigos (Walter y Zoe y Yara y Belen). Modifica el código para garantizar esto.

In [126]:
import random
from copy import deepcopy

### **Paso 1: Configuro los parametros del AG**

In [127]:
# Lista de alumnos y sus amigos (input original)
alumnos = list(alumnos_amigos.keys())
cantidad_alumnos = len(alumnos)
cantidad_cursos = 3

In [128]:
# Capacidad por curso (distribución equitativa con resto)
base = cantidad_alumnos // cantidad_cursos
resto = cantidad_alumnos % cantidad_cursos
capacidades = [base + (1 if i < resto else 0) for i in range(cantidad_cursos)]

In [129]:
# Parámetros del algoritmo genético
population_size = 100
generations = 500
mutation_rate = 0.1
tournament_size = 3
elitism = True

In [130]:
# Alumnos que deben estar en el mismo curso sí o sí
fixed_block = {'Ana': 3, 'Martin': 2, 'Horacio': 1}
block_indices = [alumnos.index(name) for name in fixed_block]

### **Paso 2: Create Population**

In [ ]:
def create_individual():
    """Crea un individuo válido respetando el bloque forzado y capacidades."""
    cursos = capacidades.copy()
    block_course = random.randint(0, cantidad_cursos - 1)
    cursos[block_course] -= len(block_indices)
    
    individual = [None] * cantidad_alumnos
    for idx in block_indices:
        individual[idx] = block_course

    restantes = []
    for c in range(cantidad_cursos):
        restantes += [c] * cursos[c]
    random.shuffle(restantes)

    j = 0
    for i in range(cantidad_alumnos):
        if individual[i] is None:
            individual[i] = restantes[j]
            j += 1
    return individual

### **Paso 3: Fitness Function**

In [132]:
def fitness(individual):
    """Calcula la utilidad total según las reglas de amistad y penalizaciones."""
    total = 0
    for i, alumno in enumerate(alumnos):
        curso = individual[i]
        amigo1, amigo2 = alumnos_amigos[alumno]
        i1, i2 = alumnos.index(amigo1), alumnos.index(amigo2)
        c1, c2 = individual[i1], individual[i2]
        if c1 == curso: total += 1
        if c2 == curso: total += 1
        if c1 == c2 and c1 != curso: total -= 3
        if curso != c1 and curso != c2 and c1 != c2: total -= 1
    return total

### **Paso 4: Parent Selection**

In [133]:
def tournament_selection(population, fitnesses):
    """Selecciona al mejor individuo de un torneo de k aleatorios."""
    participants = random.sample(list(zip(population, fitnesses)), tournament_size)
    participants.sort(key=lambda x: x[1], reverse=True)
    return deepcopy(participants[0][0])

### **Paso 5: Cruza (crossover) + reparación del bloque y capacidades**

In [134]:
def crossover(parent1, parent2):
    """Cruza uniformemente dos padres y repara el resultado."""
    child = [parent1[i] if random.random() < 0.5 else parent2[i] for i in range(cantidad_alumnos)]
    block_course = child[block_indices[0]]
    for idx in block_indices:
        child[idx] = block_course
    return repair(child)

def repair(child):
    """Ajusta para que cada curso tenga la cantidad justa de alumnos."""
    conteos = [child.count(c) for c in range(cantidad_cursos)]
    for c in range(cantidad_cursos):
        while conteos[c] > capacidades[c]:
            idxs = [i for i in range(cantidad_alumnos) if child[i] == c and i not in block_indices]
            idx = random.choice(idxs)
            for d in range(cantidad_cursos):
                if conteos[d] < capacidades[d]:
                    child[idx] = d
                    conteos[c] -= 1
                    conteos[d] += 1
                    break
    return child

### **Paso 6: Mutation**

In [135]:
def mutate(child):
    """Intercambia dos posiciones aleatorias (que no pertenezcan al bloque)."""
    if random.random() < mutation_rate:
        i, j = random.sample([x for x in range(cantidad_alumnos) if x not in block_indices], 2)
        child[i], child[j] = child[j], child[i]
    return child

### **Paso 7: Replacement + Main Loop**

In [136]:
# Inicializar población
population = [create_individual() for _ in range(population_size)]
fitnesses = [fitness(ind) for ind in population]

# Bucle principal de evolución
for _ in range(generations):
    new_population = []
    if elitism:
        best_idx = max(range(population_size), key=lambda i: fitnesses[i])
        new_population.append(deepcopy(population[best_idx]))
    while len(new_population) < population_size:
        p1 = tournament_selection(population, fitnesses)
        p2 = tournament_selection(population, fitnesses)
        child = crossover(p1, p2)
        child = mutate(child)
        new_population.append(child)
    population = new_population
    fitnesses = [fitness(ind) for ind in population]

### **Paso 8: Muestro el Resultado Final**

In [137]:
best = population[fitnesses.index(max(fitnesses))]
best_value = max(fitnesses)

print(f"Utilidad total mejor encontrada: {best_value}\n")
for c in range(cantidad_cursos):
    grupo = [alumnos[i] for i in range(cantidad_alumnos) if best[i] == c]
    print(f"Curso {c+1} ({len(grupo)} alumnos):")
    print(grupo, "\n")

Utilidad total mejor encontrada: 85

Curso 1 (20 alumnos):
['Diego', 'Elena', 'Francisco', 'Gabriela', 'Nicolas', 'Olivia', 'Pablo', 'Valeria', 'Walter', 'Xavier', 'Yara', 'Ricardo', 'Sabrina', 'Tadeo', 'Beltran', 'Catalina', 'Fabian', 'Hernan', 'Isidoro', 'Jimena'] 

Curso 2 (20 alumnos):
['Hugo', 'Isabel', 'Juan', 'Raul', 'Sofia', 'Tomas', 'Ursula', 'Zoe', 'Kevin', 'Laura', 'Mateo', 'Natalia', 'Octavio', 'Paula', 'Victor', 'Wanda', 'Ximena', 'Yamila', 'Dario', 'Gisela'] 

Curso 3 (20 alumnos):
['Ana', 'Bruno', 'Camila', 'Karla', 'Leonardo', 'Mariana', 'Andres', 'Belen', 'Carlos', 'Daniela', 'Esteban', 'Fernanda', 'Gustavo', 'Helena', 'Ignacio', 'Julieta', 'Emilia', 'Horacio', 'Luz', 'Martin'] 

